# <font color=dimgray>Chapter 2 ~ Housing Prices 

<font color=dimgray>Part 2 ~ Building & Tuning Machine Learning Models

<font color=red>Assumes you have completed <u>Part 1</u> ~ Data Exploration</font> 

## <font color=blue>Import needed modules</font>

In [1]:
##!pip install scikit-learn-intelex  #install patches to speed up processing (only need to run this ONCE on your computer or Colab session)
#from sklearnex import patch_sklearn #import the patches
#patch_sklearn()

In [2]:
#general libraries needed
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import randint

#scikit learn imports
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#not necessary but helps to visualize pipelines and models
from sklearn import set_config
set_config(display='diagram')

## <font color=blue>Function Definitions</font>

In [3]:
#function to verify the existence of a file in the current working directory and download it if not
import os,urllib, urllib.request, sys, tarfile
def downloadDataResource(file,sourcePath,compressed=None):
    if not os.path.isfile(file):
        try:
            urllib.request.urlretrieve(sourcePath+(compressed if compressed else file),(compressed if compressed else file))
            print("Downloaded", (compressed if compressed else file) )
            if compressed:
                ucomp = tarfile.open(compressed)
                ucomp.extractall()
                ucomp.close()
                print("File uncompressed.")
        except:
            print("ERROR: File", (compressed if compressed else file), "not found. Data source missing.")
    else:
        print("Data resource", file, "already downloaded.")

## <font color=blue>Source Data</font>

In [4]:
path = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/'
compressedfile = "housing.tgz"
filename = 'housing.csv'

#download data files if not currently downloaded into the current working directory
downloadDataResource(filename, path, compressedfile)

#create the dataframe
housing = pd.read_csv(filename)

Data resource housing.csv already downloaded.


## <font color=blue>Engineer the Data for Analysis</font>

In [5]:
#add features to our dataframe - the ratio of rooms and population per household
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["population_per_household"]=housing["population"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]

In [6]:
#create a new attribute that cuts (pandas cut function) Median Income into 5 different bins
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [7]:
#look at the size of the data set (number of rows x number of columns)
housing.shape

(20640, 14)

## <font color=blue>Create Training and Test Datasets </font>

### Create a Stratified Sampling based on <i>Medium Income

In [8]:
#by adding the stratify feature to the train_test_split, can ensure a more representative collection of data
X_train, X_test, y_train, y_test = train_test_split(
    housing.drop(columns=['median_house_value']),
    housing.median_house_value, 
    test_size=0.2, 
    random_state=42, 
    stratify=housing.income_cat
)

In [9]:
#now that we have the stratified test and training sets, no need for attribute income_cat in those sets
X_train.drop(columns=["income_cat"], inplace=True)
X_test.drop(columns=["income_cat"], inplace=True)

In [10]:
#ensure a good copy of the stratified training dataset and create our X and y for linear regression
housing = X_train.copy()        #changing name to be consistent with the book
housing_labels = y_train.copy() #changing name to be consistent with the book

#show the size of the data sets
housing.shape, housing_labels.shape, X_test.shape, y_test.shape

((16512, 12), (16512,), (4128, 12), (4128,))

In [11]:
#define variables needed to seperate numeric attributes from categorical attributes
cat_attribs = ["ocean_proximity"]
num_attribs = list( housing.drop(columns=cat_attribs) )

#list the numeric attributes
num_attribs

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'population_per_household',
 'bedrooms_per_room']

### Prepare Data using a Pipeline

In [12]:
#define pipeline for numeric attributes (this code is just a definition)
#each numeric attribute will be imputated using the Median strategy
#each numeric attribute will be scaled 

num_pipeline = Pipeline( [
 ('imputer', SimpleImputer(strategy="median")),
 ('std_scaler', StandardScaler()),   
])

In [13]:
#define transformation pipeline for both numeric and category attributes in the dataframe
#numeric attributed will be processed by the num_pipeline defined above
#the category ocean proximity will be transformed using the One Hot Encoder method

full_pipeline = ColumnTransformer( [
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(sparse=False), cat_attribs)
])

In [14]:
full_pipeline

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('std_scaler',
                                                  StandardScaler())]),
                                 ['longitude', 'latitude', 'housing_median_age',
                                  'total_rooms', 'total_bedrooms', 'population',
                                  'households', 'median_income',
                                  'rooms_per_household',
                                  'population_per_household',
                                  'bedrooms_per_room']),
                                ('cat', OneHotEncoder(sparse=False),
                                 ['ocean_proximity'])])

In [15]:
#take the training data set and create an array of prepared data
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.26758118, -0.1259716 ,  1.22045984, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ...,  0.        ,
         0.        ,  0.        ],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.        ,
         0.        ,  0.        ],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
#look at the size of the dataset
housing_prepared.shape

#the shape has changed ... the same number of rows but more columns
#there is a new column for all 11 numeric attribute 
#plus the encoded attributes for the 5 Ocean Proximity values

(16512, 16)

In [17]:
#use the pipeline transformation to prepare the test data set as well
#NOTE that for the test data, only need to transform it (i.e. apply transformations) but do not need to fit it
X_test = full_pipeline.transform(X_test)
X_test

array([[ 0.59229422, -0.71065803,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [-0.42180959, -0.35049119, -0.37006852, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.56232071, -0.64985064,  0.5842485 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.07211862, -0.56097831,  1.14093342, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.83208232, -0.93985512,  0.10708999, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.50736927, -0.67791559,  0.5842485 , ...,  0.        ,
         0.        ,  0.        ]])

### Preserve a list of all attributes used to aid in results analysis

In [18]:
#create a list of attributes   - THIS CODE IS NEW but able to be used more universally
attributes = []
for item in full_pipeline.transformers_:
    addToList=True
    if isinstance(item[1], Pipeline):
        for step in item[1]:
            if isinstance(step, OneHotEncoder) and item[2]!=[]:
                attributes+= list( step.get_feature_names_out() )
                addToList=False  
    if isinstance(item[1], OneHotEncoder):
        if item[2]!=[]:
            attributes+= list( item[1].get_feature_names_out() )
        addToList=False    
    if addToList:
        attributes += list(item[2])  
attributes

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'population_per_household',
 'bedrooms_per_room',
 'ocean_proximity_<1H OCEAN',
 'ocean_proximity_INLAND',
 'ocean_proximity_ISLAND',
 'ocean_proximity_NEAR BAY',
 'ocean_proximity_NEAR OCEAN']

## <font color=Blue> Establish a Baseline

In [19]:
#calculate the average home value
baseline_prediction = housing_labels.median()

#populate an array with the average home value
predictions = np.full(shape=len(housing_prepared), fill_value = baseline_prediction)

#determine the Root Mean Squared Error based on the actual vs. the baseline prediction
baseline_rmse = mean_squared_error(housing_labels, predictions, squared=False)
print("Baseline guess for any house: ${:,.0f}".format(baseline_prediction))
print("Baseline Performance (of this guess): RMSE ${:,.0f}".format(baseline_rmse))

Baseline guess for any house: $179,500
Baseline Performance (of this guess): RMSE $118,922


## <font color=blue>LinearRegression Model</font>

In [20]:
#create the model object
lin_reg = LinearRegression()

#fit the model to the prepared test data
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression()

### Evaluate the Model

In [21]:
#see how well the model fits

#calculate the predicted values
housing_predictions = lin_reg.predict(housing_prepared)

#compare the predicted (housing_predictions) to the actuals (housing_labels or what we often call y_test)
lin_rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)
print("Prediction Error (RMSE): ${:,.0f}".format(lin_rmse))

Prediction Error (RMSE): $68,161


<font color=red>Better than baseline ... but given most houses fall between 120k and 265k, the model is most likely underfitting!

### EvaluateModel using Cross Validation

In [22]:
#use cross valudation to process the data 10 different ways using linear regression model generated above
#helps us to understand how well the model "fits" our data
scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                         scoring="neg_root_mean_squared_error", cv=10)

# look at the results of each fold
scores

array([-71181.95098096, -63825.56512744, -67741.07241029, -68256.29685279,
       -66711.2271161 , -72240.73664717, -69981.07976999, -68438.51358706,
       -65946.92067317, -70073.85093767])

In [23]:
#calculate the average score over the 10 different cross validations
print("Average of RMSE across folds: ${:,.0f}".format(-scores.mean()))
print("Standard deviation: {:,.0f}".format( scores.std() ) )

Average of RMSE across folds: $68,440
Standard deviation: 2,409


<font color=red>Similar results with cross validation. Because there is limited tunning of the LinearRegression algorithm, it is time to try other Regression algorithms!

## <font color=blue>Decision Tree Regressor Model</font>

In [24]:
tree_reg = DecisionTreeRegressor( random_state=42 )
tree_reg.fit( housing_prepared, housing_labels)

DecisionTreeRegressor(random_state=42)

### Evaluate the Model

In [25]:
#see how well the model fits
#calcualte the predicted values
housing_predictions = tree_reg.predict(housing_prepared)

#compare the predicted (housing_predictions) to the actuals(housing_labels)
tree_rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)
print("Prediction Error (RMSE): ${:,.0f}".format(tree_rmse))

Prediction Error (RMSE): $0


<font color=red>Did we find the perfect model? No, most likely overfitting!

### Evaluate Model using Cross Validation</font>

In [26]:
# use cross valudation to evaluate/score the Decision Tree Regressor Model 5 different ways
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_root_mean_squared_error", cv=5)

#look at the results of each fold
scores

array([-68901.48374735, -70554.79310661, -68262.4982775 , -72200.95062288,
       -72274.34626073])

In [27]:
#calculate the average score over the 10 different cross validations
print("Average of RMSE across folds: ${:,.0f}".format(-scores.mean()))
print("Standard deviation: {:,.0f}".format( scores.std() ) )

Average of RMSE across folds: $70,439
Standard deviation: 1,648


<font color=red>Seems the Linear Regression model is better than Decision Tree Regressor but both are bad predictors!

## <font color=blue>Random Forest Regressor</font>

In [28]:
#create a Random Forest Regressor that uses random attributes and averages out their predictions
#Note: this may take some time because it is creating 100 trees in this forest
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)

#fit the model to the data
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(random_state=42)

### Evaluate the Model

In [29]:
#let's see how well it performs compared to Linear Regression and Decision Tree Regressor
housing_predictions = forest_reg.predict(housing_prepared)

#compare the predicted (housing_predictions) to the actuals(housing_labels)
forest_rmse = mean_squared_error(housing_labels,housing_predictions,squared=False)
print("Prediction Error (RMSE): ${:,.0f}".format(forest_rmse))

Prediction Error (RMSE): $18,656


<font color=red>A much better model! Or is it?

### Evaluate Model using Cross Validation

<font color=red>THIS WILL TAKE TIME

In [30]:
#use cross valudation to process the data 5 different ways using the random forest model generated above
scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring="neg_root_mean_squared_error", cv=5)

# look at the results of each fold
scores

array([-50081.75115812, -49694.54011604, -50123.14181907, -51606.82657621,
       -51672.39011958])

In [31]:
#calculate the average score over the 10 different cross validations
print("Average of RMSE across folds: ${:,.0f}".format(-scores.mean()))
print("Standard deviation: {:,.0f}".format( scores.std() ) )

Average of RMSE across folds: $50,636
Standard deviation: 833


<font color=red>Cross validation indicates that model tuning is necessary. The training data set performed <u>much better</u> than cross validation. Hence, the model is still overfitting.

### <font color=blue>Take best Model and Tune using GridSearch Cross Validation</font>

 <font color=red>THIS WILL TAKE TIME

In [32]:
# Best Model was RandomForestRegressor - will now tune its hyperparameters

# create a new Random Forest Regressor 
forest_reg = RandomForestRegressor( random_state=42 )

#create a parameter grid that determines the variable hyperparameters
#try 12 (3×4) combinations of hyperparameters
#and then then try 6 (2×3) combinations with bootstrap set as False
param_grid = [   
    {'n_estimators': [3, 10, 30], 'max_features': [4, 6, 8, 10]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [4, 6, 8]},
  ]

# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_root_mean_squared_error',
                           return_train_score=True)

#fit the model to the data
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [4, 6, 8, 10],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [4, 6, 8],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [33]:
#let's look at the score of each hyperparameter combination tested during the grid search
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(-mean_score, params)

60905.32379928889 {'max_features': 4, 'n_estimators': 3}
52825.53539272015 {'max_features': 4, 'n_estimators': 10}
50279.91335864639 {'max_features': 4, 'n_estimators': 30}
57554.03887670543 {'max_features': 6, 'n_estimators': 3}
51482.43455956665 {'max_features': 6, 'n_estimators': 10}
49767.45433403617 {'max_features': 6, 'n_estimators': 30}
59682.414248445726 {'max_features': 8, 'n_estimators': 3}
52766.05223811932 {'max_features': 8, 'n_estimators': 10}
49921.80892278787 {'max_features': 8, 'n_estimators': 30}
58532.62514659994 {'max_features': 10, 'n_estimators': 3}
52333.58265785854 {'max_features': 10, 'n_estimators': 10}
50216.62753828196 {'max_features': 10, 'n_estimators': 30}
58372.99449769822 {'bootstrap': False, 'max_features': 4, 'n_estimators': 3}
51137.26385288084 {'bootstrap': False, 'max_features': 4, 'n_estimators': 10}
57558.927791430135 {'bootstrap': False, 'max_features': 6, 'n_estimators': 3}
51261.14203158512 {'bootstrap': False, 'max_features': 6, 'n_estimators

In [34]:
#display the parameters with the best solution (lowest score)
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [35]:
#display the best model and its parameters (similar to above but not in dictionary format)
grid_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=30, random_state=42)

### <font color=blue>Tune the same model using Randomized Search Cross Validation</font> 

<font color=red>THIS WILL TAKE TIME

In [36]:
# continue to use the random forest regressor but randomly have hyperparameters assigned

#create a parameter grid that uses random hyperparameters
param_distribs = {
        'n_estimators': randint(low=100, high=200),  #Note the book uses 1 as the low but to save time, changed to 100
        'max_features': randint(low=1, high=10),
    }

# train across 5 folds 
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_root_mean_squared_error', random_state=42)

rnd_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001E2849D1640>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001E2849CAC40>},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [37]:
#let's look at the score of each hyperparameter combination tested during the grid search
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(-mean_score, params)

49202.001027100996 {'max_features': 7, 'n_estimators': 151}
49215.620395947735 {'max_features': 8, 'n_estimators': 160}
49247.47458878027 {'max_features': 5, 'n_estimators': 182}
49141.08397093201 {'max_features': 7, 'n_estimators': 174}
49153.72388993532 {'max_features': 8, 'n_estimators': 199}
49221.70827554076 {'max_features': 8, 'n_estimators': 123}
50314.23908486547 {'max_features': 3, 'n_estimators': 121}
49505.73497826164 {'max_features': 5, 'n_estimators': 101}
49212.9971781265 {'max_features': 8, 'n_estimators': 129}
49227.2525868698 {'max_features': 6, 'n_estimators': 101}


In [38]:
#list the parameters the random search found to be the best
rnd_search.best_params_

{'max_features': 7, 'n_estimators': 174}

### Confirm that Tuning Produces a Better Result  

<font color=red>THIS WILL TAKE TIME

In [39]:
# create a new Random Forest Regressor with the best parameters  
forest_reg = RandomForestRegressor(**rnd_search.best_params_, random_state=42)
scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring="neg_root_mean_squared_error", cv=5)

# look at the results of each fold
scores

array([-49127.92809937, -48296.15191449, -48063.78140575, -49631.24824523,
       -50586.31018982])

In [40]:
print("Average of RMSE across folds: ${:,.0f}".format(-scores.mean()))
print("Standard deviation: {:,.0f}".format( scores.std() ) )

Average of RMSE across folds: $49,141
Standard deviation: 917


#### The best model results so far!

### <font color=blue>Analyze Feature Importance

In [41]:
#still working with the Random Forest Regressor model as it it is "the best" performing model

#get the features importance from the last random CV
feature_importances = rnd_search.best_estimator_.feature_importances_
feature_importances

array([7.08332485e-02, 6.41611629e-02, 4.34835623e-02, 1.63834877e-02,
       1.57201462e-02, 1.60205209e-02, 1.52522766e-02, 3.40594035e-01,
       5.97790861e-02, 1.10543409e-01, 7.24146564e-02, 9.10871538e-03,
       1.58986256e-01, 7.77987535e-05, 2.94732290e-03, 3.69431512e-03])

<font color=red>The values tell us nothing without the associated attributes.

In [42]:
#list attributes with their importance (as determined by the model)
sorted(zip(feature_importances, attributes), reverse=True)

[(0.34059403452409215, 'median_income'),
 (0.15898625646709086, 'ocean_proximity_INLAND'),
 (0.11054340934748115, 'population_per_household'),
 (0.07241465638050859, 'bedrooms_per_room'),
 (0.07083324851774955, 'longitude'),
 (0.06416116289932089, 'latitude'),
 (0.05977908606487608, 'rooms_per_household'),
 (0.04348356225643111, 'housing_median_age'),
 (0.016383487663536384, 'total_rooms'),
 (0.01602052088989179, 'population'),
 (0.015720146212569126, 'total_bedrooms'),
 (0.015252276618024613, 'households'),
 (0.00910871538470401, 'ocean_proximity_<1H OCEAN'),
 (0.003694315119938403, 'ocean_proximity_NEAR OCEAN'),
 (0.0029473229002742146, 'ocean_proximity_NEAR BAY'),
 (7.77987535111078e-05, 'ocean_proximity_ISLAND')]

<font color=red>From this we can see that getting rid of the last 3 attributes (and maybe even more) would have little impact on the model.

### <font color=blue>Evaluate on the Test Set

In [43]:
#taking the best model so far and making it our final model
final_model = rnd_search.best_estimator_

In [44]:
#predict the test data set
predictions = final_model.predict(X_test)

#measure performance
rmse = mean_squared_error(y_test, predictions, squared=False)
print("TEST Predition Error (RMSE): ${:,.0f}".format(rmse))

TEST Predition Error (RMSE): $46,999


In [45]:
#We can compute a 95% confidence interval for the test RMSE
confidence = 0.95
squared_errors = (predictions - y_test) ** 2
confidence_interval = np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, 
                                               loc=squared_errors.mean(), scale=stats.sem(squared_errors)))
print( "{:.0f}% Confidence Intervals: {:,.2f} -- {:,.2f}".format(confidence*100, confidence_interval[0], confidence_interval[1]) )

95% Confidence Intervals: 45,022.52 -- 48,896.55
